In [0]:
from pyspark.sql.functions import col

#READ FROM STREAM AND CREATE BRONZE CAR
writeBronzeTable = (spark
                        .readStream
                        .format("delta")
                        .load("/tmp/dbldatagen/carDatatry/data")
                        .writeStream
                        .format("delta")
                        .outputMode("append")
                        .option("checkpointLocation", "/tmp/delta/demoCheckpointTable/checkpoints_bronze_car/")
                        .toTable("default.bronze_car"))


#READ FROM STREAM AND CREATE BRONZE SOUND
writeBronzeTableSound = (spark
                        .readStream
                        .format("delta")
                        .load("/tmp/dbldatagen/soundDatatry/data")
                        .writeStream
                        .format("delta")
                        .outputMode("append")
                        .option("checkpointLocation", "/tmp/delta/demoCheckpointTable/checkpoints_bronze_sound/")
                        .toTable("default.bronze_sound"))

#CLEAN BRONZE AND PREPARE SILVER CAR
silverCarDF = (spark
                .readStream
                .format("delta")
                .table("default.bronze_car")
                .where(col('car_color')!='NaN')
                .where(col('car_type')!='NaN')
                .where(col('car_brand')!='NaN')
                .where(col('time').isNotNull()))

#CLEAN BRONZE AND PREPARE SILVER SOUND
silverSoundDF = (spark
                .readStream
                .format("delta")
                .table("default.bronze_sound")
                .where(col('decibels') >= 0)
                .where(col('time').isNotNull()))

#CREATE IF NOT EXIST AND UPDATE SILVER TABLE Car
writeSilverCarTable = (silverCarDF
                        .writeStream
                        .format("delta")
                        .outputMode("append")
                        .option("checkpointLocation", "/tmp/delta/demoCheckpointTable/checkpoints_silver_car/")
                        .toTable("default.silver_car"))

#CREATE IF NOT EXIST AND UPDATE SILVER TABLE SOUND
writeSilverSoundTable = (silverSoundDF
                        .writeStream
                        .format("delta")
                        .outputMode("append")
                        .option("checkpointLocation", "/tmp/delta/demoCheckpointTable/checkpoints_silver_sound/")
                        .toTable("default.silver_sound"))

cityCoordsDF = spark.read.table("default.city_coords")

#CREATE IF NOT EXIST AND UPDATE GOLD TABLE CAR
writeGoldCarTable = (spark
                        .readStream
                        .format("delta")
                        .table("default.silver_car")
                        .join(cityCoordsDF, "city", "inner")
                        .writeStream
                        .format("delta")
                        .outputMode("append")
                        .option("checkpointLocation", "/tmp/delta/demoCheckpointTable/checkpoints_gold_car/")
                        .toTable("default.gold_car"))

#CREATE IF NOT EXIST AND UPDATE GOLD TABLE SOUND
writeGoldSoundTable = (spark
                        .readStream
                        .format("delta")
                        .table("default.silver_sound")
                        .join(cityCoordsDF, "city", "inner")
                        .writeStream
                        .format("delta")
                        .outputMode("append")
                        .option("checkpointLocation", "/tmp/delta/demoCheckpointTable/checkpoints_gold_sound/")
                        .toTable("default.gold_sound"))



In [0]:
#clean all


# dbutils.fs.rm('/user/hive/warehouse/bronze_car',True)
# dbutils.fs.rm('/user/hive/warehouse/silver_car',True)
# dbutils.fs.rm('/user/hive/warehouse/gold_car',True)


# dbutils.fs.rm('/user/hive/warehouse/bronze_sound',True)
# dbutils.fs.rm('/user/hive/warehouse/silver_sound',True)
# dbutils.fs.rm('/user/hive/warehouse/gold_sound',True)

#dbutils.fs.rm('/user/hive/warehouse/city_coords',True)
#dbutils.fs.rm('/user/hive/warehouse/historical_car',True)
#dbutils.fs.rm('/user/hive/warehouse/historical_sound',True)


In [0]:
# cityCoordsDF = spark.read.table("default.city_coords")

# #WRITE HOSTIRICAL FROM S3 STORAGE
# writeHistorical = (spark.read
#                    .parquet(f'/mnt/bucket-big-traffic/big_data_prj/demo_example/big_car')
#                    .join(cityCoordsDF, "city", "inner")
#                    .write
#                    .format("delta")
#                    .mode("append")
#                    .saveAsTable("default.historical_car"))

# writeSound = (spark.read
#                    .parquet(f'/mnt/bucket-big-traffic/big_data_prj/demo_example/big_sound')
#                    .join(cityCoordsDF, "city", "inner")
#                    .write
#                    .format("delta")
#                    .mode("append")
#                    .saveAsTable("default.historical_sound"))